In [1]:
#https://www.youtube.com/watch?v=1LCb1PVqzeY

In [2]:
import cv2
import numpy as np


In [3]:
#  https://pjreddie.com/darknet/yolo/ to download weight file
#!git clone https://github.com/pjreddie/darknet to download cfg file
#https://github.com/pjreddie/darknet/blob/master/data/coco.names
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

In [4]:
classes=[]
with open('coco.names','r') as f:
    classes = f.read().splitlines()

print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [5]:
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

out = cv2.VideoWriter("outputYOLO.mp4", fourcc, 5.0, (1280,720))

In [6]:
#from video
cap = cv2.VideoCapture('1.mp4')



In [7]:
#1/255 image normailzation 
#416,416 image resize
#swapRB change position red and blue (rgb to bgr)

while True:
    _, img= cap.read()
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255,  (416,416), (0,0,0), swapRB=True, crop=False)

    # for b in blob:
    #     for n, img_blob in enumerate(b):
    #         cv2.imshow()

    #to set input 
    net.setInput(blob)

    #get output layers names
    output_layers_names = net.getUnconnectedOutLayersNames()

    #get function layerOutputs provided by layer names
    layerOutputs = net.forward(output_layers_names)

    boxes=[]
    confidences = []
    class_ids =[]

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 :
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    print(len(boxes))

    #0.5 is same with confidence threshold
    #0.4 is max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    #colors = np.random.uniform(0,255,size=(len(boxes), 3))
    if len(indexes)>0:
        for i in indexes.flatten():
            
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            if label == 'person':
                confidence = str(round(confidences[i],2))
                #color = colors[i]
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
                cv2.putText(img, label + " "+ confidence, (x,y+20), font, 2, (255,255,255), 2)
            
    
    #esc key
    cv2.imshow('Image', img)
    imageWrite = cv2.resize(img, (1280,720))
    out.write(imageWrite)
    key = cv2.waitKey(1)
    if key==27:
        break

    

28
25
28
29
30
32
31
32
32
33
31
30
29
27
30
32
29
31
28
26
26
26
27
24
20
21
23
25
23
24
22
24
25
27
26
25
23
28
28
30
30
30
32
32
33
33
34
32
31
31
30
30
27
26
26
27
27
28
26
27
27
27
26
30
35
33
33
35
35
36
39
40
39
40
36
33
31
27
27
27
28
28
28
31
32
34
31
33
34
36
33
33
33
33
35
38
33
36
34
34
35
37
35


In [8]:
cap.release()
out.release()
cv2.destroyAllWindows()
